# Approche

La technique d'apprentissage par transfert est utilisée pour résoudre le problème de classification de texte. Nous chargeons un modèle BERT préentraîné et ajustons ses poids.
Avantages de l'ajustement fin

  - Temps - Les poids du modèle BERT préentraîné encodent déjà beaucoup d’informations. Par conséquent, il faut beaucoup moins de temps pour affiner le modèle.

  - Données - Étant donné que le modèle préentraîné a été entraîné sur un large corpus de texte, il donne de bons résultats même avec de petits ensembles de données.

Nous n’entrons pas dans les détails de l’architecture de BERT. Voici un aperçu de la façon dont BERT est préentraîné et comment il peut être utilisé pour la classification.

## BERT (Bidirectional Encoder Representations from Transformers)

Le modélisation du langage est une méthode courante de pré-entraînement sur du texte non annoté (apprentissage auto-supervisé). La plupart des modèles de langage sont entraînés en prédisant de manière itérative le mot suivant dans une séquence, de manière auto-régressive, sur d'immenses ensembles de données textuelles comme Wikipédia. Cette prédiction peut se faire de gauche à droite, de droite à gauche ou de manière bidirectionnelle.

Il existe deux stratégies pour appliquer des représentations de langage pré-entraînées à des tâches en aval :

* Approche basée sur les caractéristiques (Feature-based approach)
* Approche d'affinage (Fine-tuning approach)

L'approche basée sur les caractéristiques, comme ELMo, utilise des architectures spécifiques à la tâche qui incluent les représentations pré-entraînées en tant que caractéristiques supplémentaires.

L'approche d'affinage, comme OpenAI GPT, introduit un minimum de paramètres spécifiques à la tâche et est entraînée sur la tâche cible en affinant tous les paramètres pré-entraînés.

Le modèle BERT peut être utilisé pour les deux approches. Il reformule la tâche de pré-entraînement en modélisation du langage, qui consistait à prédire itérativement le mot suivant dans une séquence, pour au contraire intégrer un contexte bidirectionnel et prédire des tokens masqués intermédiaires dans la séquence. BERT a introduit une nouvelle tâche d'apprentissage auto-supervisé pour pré-entraîner les Transformers afin de les affiner pour différentes tâches. La principale différence entre BERT et les méthodes précédentes de pré-entraînement des modèles Transformers est l'utilisation du contexte bidirectionnel de la modélisation du langage. La plupart des modèles prédisent le mot suivant en se déplaçant soit de gauche à droite, soit de droite à gauche, alors que BERT apprend à prédire des tokens intermédiaires (grâce au mécanisme de MASK), d'où son nom de Bidirectional Encoder.

BERT utilise deux tâches d'apprentissage :

* Le modèle de langage masqué (Masked Language Model - MLM)
* La prédiction de la phrase suivante (Next Sentence Prediction - NSP)

BERT utilise trois types d'embeddings pour calculer les représentations d'entrée :

* Token embeddings
* Segment embeddings
* Position embeddings

Le Transformer de BERT conserve la longueur (ou la dimension) de l'entrée. La sortie finale prend ce vecteur et l'achemine vers différentes tâches, comme la classification dans ce cas.

## Installation des librairies nécessaires

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import numpy as np
import pandas as pd
import time
import datetime
import random
import preprocessing as pp
from tqdm import tqdm

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
data_train = pd.read_csv("./datasets/X_y_train.csv")
df_cleaned = data_train["text"].apply(lambda x : pp.remove_noise(x))
y=data_train["level"]

# Data preprocessing

En utilisant la librairie fournie par le challenge kaggle, preprocessing.py, nous appliquons un prétraitement pour enlever tout ce qui pourrait perturber la classification du modèle comme `\n`.

### BERT Tokenizer

De plus, nous appliquons l'algorithme de tokenization de BERT, qui se nomme WordPiece, pour transformer les textes en suites d'indice de tokens.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Un exemple

print(' Original: ', df_cleaned[0])

print('Tokenized: ', tokenizer.tokenize(df_cleaned[0]))

print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(df_cleaned[0])))

 Original:  Hi! Just some words about the product With the cost is in the average Market. The important differences are the features. a which could translatesuseful phrases intolanguages as the concurent But futhermore with out product we have a curruncy converter, headphone set, world radio, pedometer. Besides it is less heavy. An other concurrent has the advantage being smaller than ours but its more expensive and less of features. A big advantage, our product listens to your pronunciation and corrects inlanguages. I think we could logical saying is art. Best Regards. Bob
Tokenized:  ['hi', '!', 'just', 'some', 'words', 'about', 'the', 'product', 'with', 'the', 'cost', 'is', 'in', 'the', 'average', 'market', '.', 'the', 'important', 'differences', 'are', 'the', 'features', '.', 'a', 'which', 'could', 'translates', '##use', '##ful', 'phrases', 'into', '##lang', '##ua', '##ges', 'as', 'the', 'con', '##cure', '##nt', 'but', 'fu', '##ther', '##more', 'with', 'out', 'product', 'we', 'have

In [ ]:
max_len = 0

for sent in df_cleaned:

    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  509


In [ ]:
input_ids = []
attention_masks = []

for sent in df_cleaned:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = max_len,      
                        pad_to_max_length = True,
                        return_attention_mask = True, 
                        return_tensors = 'pt',     
                   )

    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(y)

print('Original: ', df_cleaned[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  Hi! Just some words about the product With the cost is in the average Market. The important differences are the features. a which could translatesuseful phrases intolanguages as the concurent But futhermore with out product we have a curruncy converter, headphone set, world radio, pedometer. Besides it is less heavy. An other concurrent has the advantage being smaller than ours but its more expensive and less of features. A big advantage, our product listens to your pronunciation and corrects inlanguages. I think we could logical saying is art. Best Regards. Bob
Token IDs: tensor([  101,  7632,   999,  2074,  2070,  2616,  2055,  1996,  4031,  2007,
         1996,  3465,  2003,  1999,  1996,  2779,  3006,  1012,  1996,  2590,
         5966,  2024,  1996,  2838,  1012,  1037,  2029,  2071, 16315,  8557,
         3993, 15672,  2046, 25023,  6692,  8449,  2004,  1996,  9530, 23887,
         3372,  2021, 11865, 12399,  5974,  2007,  2041,  4031,  2057,  2031,
         1037, 1273

#### Train-validation split
80% des données sont réservées pour l'entraînement et le reste pour la validation.

In [ ]:

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.8 * len(dataset))
val_size = len(dataset)  - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

10,696 training samples
2,675 validation samples


In [12]:
batch_size = 32

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
# Charger BertForSequenceClassification, le modèle BERT pré-entraîné avec une seule
# couche linéaire de classification au-dessus.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 5,
    output_attentions = False,
    output_hidden_states = False
)

model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Affinage du modèle

In [15]:
epochs = 2
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [16]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [17]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

total_t0 = time.time()

for epoch_i in range(1, epochs+1):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in tqdm(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = output.loss
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)

    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, f'./bert_model_{epoch_i}.pt')
        best_eval_accuracy = avg_val_accuracy
    training_stats.append(
        {
            'epoch': epoch_i,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...


335it [16:01,  2.87s/it]



  Average training loss: 0.70
  Training epcoh took: 0:16:02

Running Validation...
  Accuracy: 0.86

======== Epoch 2 / 2 ========
Training...


335it [16:02,  2.87s/it]



  Average training loss: 0.33
  Training epcoh took: 0:16:02

Running Validation...
  Accuracy: 0.89

Training complete!
Total training took 0:34:49 (h:mm:ss)


# Chargement de modèle

In [21]:
model = torch.load('./bert_model_1.pt')

<ipython-input-21-c2dd83bf0fa5>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('./bert_model_1.pt')


In [25]:
df_test = pd.read_csv('./datasets/X_test.csv')
df_test['text'] = df_test['text'].apply(lambda x:pp.remove_noise(x))
test_df_cleaned = df_test['text'].values

In [26]:
test_input_ids = []
test_attention_masks = []
for sent in test_df_cleaned:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = max_len,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(
            test_dataset, 
            sampler = SequentialSampler(test_dataset), 
            batch_size = batch_size 
        )

In [28]:
predictions = []
for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()

            predictions.extend(list(pred_flat))

In [ ]:
df_output = pd.DataFrame(predictions,columns=["target"])
df_output_final = pd.concat((df_test['Id'],df_output),ignore_index=True)
df_output_final.to_csv('submission_bert.csv',index=False)